In [21]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [97]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable


from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df_train = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\train.csv')
df_test = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\test.csv')

In [98]:
table = PrettyTable()
table.field_names = ['Variable_name', 'nUnique', 'DTypes']


for i in df_train.columns:
    var = i 
    nunique = df_train[i].nunique()
    dtyp = df_train[i].dtypes
    table.add_row([var, nunique, dtyp])


table.add_column('Work Status',['DROPPED', 'later', 'later', 'NUMERICAL', 'NUMERICAL', 'CATEGORICAL', 'later', 'later', 'later', 'later','NUMERICAL', 'NUMERICAL', 'Y'])
print(table)
## Based on the information from table below, classify:
# id - Drop column
# y = Price
# Numeric variables: Model_year, milage, accident(make boolean), clean_title(make boolean)
# Categorical variables: fuel_type


# Model + brand: segment based on average price maybe??
# transmission
# Engine: super complex, separate into various columns

# ignore for now: int_col / ext_col

+---------------+---------+--------+-------------+
| Variable_name | nUnique | DTypes | Work Status |
+---------------+---------+--------+-------------+
|       id      |  188533 | int64  |   DROPPED   |
|     brand     |    57   | object |    later    |
|     model     |   1897  | object |    later    |
|   model_year  |    34   | int64  |  NUMERICAL  |
|     milage    |   6651  | int64  |  NUMERICAL  |
|   fuel_type   |    7    | object | CATEGORICAL |
|     engine    |   1117  | object |    later    |
|  transmission |    52   | object |    later    |
|    ext_col    |   319   | object |    later    |
|    int_col    |   156   | object |    later    |
|    accident   |    2    | object |  NUMERICAL  |
|  clean_title  |    1    | object |  NUMERICAL  |
|     price     |   1569  | int64  |      Y      |
+---------------+---------+--------+-------------+


In [99]:
### Drop ID column
df_train.drop(columns='id', inplace=True)
df_test.drop(columns='id', inplace=True)

In [100]:
    ### Clean the 'Accident' column
df_train['accident'].unique()
df_train.loc[df_train['accident'] == 'None reported', 'accidentbool'] = False
df_train.loc[df_train['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_train.drop(columns='accident', inplace = True)

df_test.loc[df_test['accident'] == 'None reported', 'accidentbool'] = False
df_test.loc[df_test['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_test.drop(columns='accident', inplace = True)

C:\Users\higaj\AppData\Local\Temp\ipykernel_3808\3880557373.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train['accident'] == 'None reported', 'accidentbool'] = False
C:\Users\higaj\AppData\Local\Temp\ipykernel_3808\3880557373.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_test.loc[df_test['accident'] == 'None reported', 'accidentbool'] = False


In [101]:
### Clean the 'clean_title' column
df_train['clean_title'].unique()
df_train.loc[df_train['clean_title'] == 'Yes', 'cleanbool'] = True
df_train.drop(columns='clean_title', inplace = True)

df_test.loc[df_test['clean_title'] == 'Yes', 'cleanbool'] = True
df_test.drop(columns='clean_title', inplace = True)


C:\Users\higaj\AppData\Local\Temp\ipykernel_3808\774671184.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train['clean_title'] == 'Yes', 'cleanbool'] = True
C:\Users\higaj\AppData\Local\Temp\ipykernel_3808\774671184.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_test.loc[df_test['clean_title'] == 'Yes', 'cleanbool'] = True


In [102]:
# Define the Tables
X = df_train
X = X.drop(columns='price')
y = df_train.price

# Define the columns that are going to be used
numerical_cols = ['model_year', 'milage']
categorical_cols = ['fuel_type', 'cleanbool', 'accidentbool']

# Separate test and validation of the df_train dataframe
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Defining strategies for numerical and categorical columns
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle the preprocessor

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Modeling
model = RandomForestRegressor(n_estimators = 100, random_state=0)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                  ('model', model)
                 ])

clf.fit(X_train, y_train)
preds = clf.predict(X_valid)
print(np.sqrt(mean_squared_error(preds, y_valid)))

76432.4042817347


In [105]:
## Run model and make it spit the submissioN!
X = df_test
prediccion = clf.predict(X)

submission = pd.DataFrame()
temp = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\sample_submission.csv')
submission['id'] = temp['id']
submission['price'] = prediccion
submission.to_csv('submission.csv',index=False)